In [25]:
import os
os.getcwd()

'C:\\Users\\Administrator\\Desktop\\数据挖掘\\数据挖掘比赛\\ccf-隐私-deepshare\\ch5'

# 信息和标签增强

In [29]:
import os
import pandas as pd
from tqdm import tqdm
import jieba
import jieba.posseg as pseg

In [30]:
def get_line(data_path):
    with open(data_path,'r',encoding="utf-8") as f:
        line = ''.join(f.readlines())
    return line

In [31]:
def get_df_label(label_path):
    return pd.read_csv(label_path,encoding="utf-8")

In [32]:
line = get_line('./data/train_data/0.txt')

In [33]:
label = get_df_label('./data/train_label/0.csv')

In [34]:
line

'艺术是相同的，音乐美术体育三样都是艺术。，三样艺术都是靠感觉的。感觉好玩起来就很轻松，所以叫做玩艺术。没感觉找不到北的干脆别玩了！，香港电影国语配音名家周思平，代表作有TVB《上海滩》周润发等'

In [35]:
label

,ID,Category,Pos_b,Pos_e,Privacy
0,0,position,66,75,香港电影国语配音名家
1,0,name,76,78,周思平
2,0,movie,87,91,《上海滩》
3,0,name,92,94,周润发
4,0,organization,84,86,TVB


In [36]:
words=pseg.cut(line)

In [37]:
for word, flag in words:
    print(word, flag)

艺术 n
是 v
相同 d
的 uj
， x
音乐 n
美术 n
体育 vn
三样 m
都 d
是 v
艺术 n
。 x
， x
三样 m
艺术 n
都 d
是 v
靠 v
感觉 n
的 uj
。 x
感觉 n
好玩 v
起来 v
就 d
很 d
轻松 a
， x
所以 c
叫做 v
玩 v
艺术 n
。 x
没 v
感觉 n
找 v
不到 v
北 f
的 uj
干脆 d
别 d
玩 v
了 ul
！ x
， x
香港电影 n
国语 nz
配音 n
名家 n
周思平 nr
， x
代表作 n
有 v
TVB eng
《 x
上海滩 ns
》 x
周润发 nr
等 u


In [40]:
def get_df_label(label_path):
    return pd.read_csv(label_path)

In [51]:
label=get_df_label("./data/train_label/0.csv")
label

,ID,Category,Pos_b,Pos_e,Privacy
0,0,position,66,75,香港电影国语配音名家
1,0,name,76,78,周思平
2,0,movie,87,91,《上海滩》
3,0,name,92,94,周润发
4,0,organization,84,86,TVB


In [45]:
#将数据分别打标成 3种标注体系 BIES BI IE
def get_list_data_ner(data_path):
    with open(data_path,'r',encoding="utf-8") as f:
        line = ''.join(f.readlines())
        words = pseg.cut(line)
        list_data = []
        list_ner = []
        list_BIES = []
        list_BI = []
        list_IE = []
        for word, flag in words:
            list_word = list(word)
            len_word = len(list_word)
            list_data += list_word
            list_ner += [flag] * len_word
            if len_word == 1:
                list_BIES += ['S']
                list_BI += ['B']
                list_IE += ['E']
            elif len_word == 2:
                list_BIES += ['B','E']
                list_BI += ['B','I']
                list_IE += ['I','E']
            else:
                list_BIES += ['B'] + ['I'] * (len_word-2) + ['E']
                list_BI += ['B'] + ['I'] * (len_word-1)
                list_IE += ['I'] * (len_word-1) + ['E']
                list_data = list(line)
    list_data_info = list_data,list_ner,list_BIES,list_BI,list_IE
    return list_data_info

In [48]:
list_data_info=get_list_data_ner('./data/train_data/0.txt')
print(list_data_info[0][0],list_data_info[1][0],list_data_info[2][0],list_data_info[3][0],list_data_info[4][0])
print(list_data_info[0][1],list_data_info[1][1],list_data_info[2][1],list_data_info[3][1],list_data_info[4][1])

艺 n B B I
术 n E I E


In [53]:
#将3种标注体系 BIES BI IE的数据标签和实体标签进行结合，拼接
def get_label_ner(list_data_info, df_label):
    list_data,list_ner,list_BIES,list_BI,list_IE = list_data_info
    list_BIOES_label = ['label_O'] * len(list_data)
    list_BIO_label = ['label_O'] * len(list_data)
    list_IEO_label = ['label_O'] * len(list_data)
    list_IO_label = ['label_O'] * len(list_data)
    for index,d in df_label.iterrows():
        ID = d['ID']
        Category = d['Category']
        Pos_b = d['Pos_b']
        Pos_e = d['Pos_e']
        
        # 异常数据处理
        if 2162 == ID:
            Category == '前明骏女孩组合队长"'
            Pos_e = Pos_e - 1

        if Pos_b == Pos_e:
            list_BIOES_label[Pos_b] = 'label_S_' + Category
            
            list_BIO_label[Pos_b] = 'label_B_' + Category
            
            list_IEO_label[Pos_b] = 'label_I_' + Category
            
            list_IO_label[Pos_b] = 'label_I_' + Category
        elif Pos_e - Pos_b == 1:
            list_BIOES_label[Pos_b] = 'label_B_' + Category
            list_BIOES_label[Pos_e] = 'label_E_' + Category
            
            list_BIO_label[Pos_b] = 'label_B_' + Category
            list_BIO_label[Pos_e] = 'label_I_' + Category
            
            list_IEO_label[Pos_b] = 'label_I_' + Category
            list_IEO_label[Pos_e] = 'label_E_' + Category
            
            list_IO_label[Pos_b] = 'label_I_' + Category
            list_IO_label[Pos_e] = 'label_I_' + Category   
        else:
            # 异常数据处理
            try:
                list_BIOES_label[Pos_b] = 'label_B_' + Category
                list_BIOES_label[Pos_e] = 'label_E_' + Category 
                for pos_i in range(Pos_b+1,Pos_e):
                    list_BIOES_label[pos_i] = 'label_I_' + Category 
                    
                list_BIO_label[Pos_b] = 'label_B_' + Category
                for pos_i in range(Pos_b+1,Pos_e+1):
                    list_BIO_label[pos_i] = 'label_I_' + Category 
                    
                list_IEO_label[Pos_e] = 'label_E_' + Category 
                for pos_i in range(Pos_b,Pos_e):
                    list_IEO_label[pos_i] = 'label_I_' + Category 
                    
                for pos_i in range(Pos_b,Pos_e+1):
                    list_IO_label[pos_i] = 'label_I_' + Category 
            except:
                print(ID)
    
    data_label_info = [(ID,data,label_BIOES,label_BIO,label_IEO,label_IO,\
                           data_ner,data_BIES,data_BI,data_IE) for data,label_BIOES,label_BIO,label_IEO,label_IO,\
                           data_ner,data_BIES,data_BI,data_IE in zip(list_data,
                                                                     list_BIOES_label,
                                                                     list_BIO_label,
                                                                     list_IEO_label,
                                                                     list_IO_label,
                                                                     list_ner,
                                                                     list_BIES,
                                                                     list_BI,
                                                                     list_IE)]
    return data_label_info

In [66]:
data_label_info=get_label_ner(list_data_info,label)
data_label_info[0],data_label_info[87]

((0, '艺', 'label_O', 'label_O', 'label_O', 'label_O', 'n', 'B', 'B', 'I'),
 (0,
  '《',
  'label_B_movie',
  'label_B_movie',
  'label_I_movie',
  'label_I_movie',
  'x',
  'S',
  'B',
  'E'))

In [72]:
def get_data_label(path_dir_data,path_dir_label):
    
    # 训练数据集
    list_data_label_info = []
    # 获取训练集 ID
    list_data_ID = [file_name.split('.')[0] for file_name in os.listdir(path_dir_data) if '.txt' in file_name]
    list_label_ID = [file_name.split('.')[0] for file_name in os.listdir(path_dir_label) if '.csv' in file_name]
    
    # 循环训练集 ID
    for ID in tqdm(set(list_data_ID) & set(list_label_ID)):
        # 获取词和标签地址
        data_path = os.path.join(path_dir_data,ID+'.txt')
        label_path = os.path.join(path_dir_label,ID+'.csv')
        
        # 获取词和标签数据
        list_data_info = get_list_data_ner(data_path)
        df_label = get_df_label(label_path)
        
        # BIOES 进行标注
        data_label_info = get_label_ner(list_data_info, df_label)
        
        # 加入训练数据集
        list_data_label_info += [data_label_info]
        
    return list_data_label_info

In [75]:
path_dir_data = './data/train_data/'
path_dir_label = './data/train_label/'

In [76]:
list_data_label_info = get_data_label(path_dir_data,path_dir_label)

100%|████████████████████████████████████████████████████████████████████████████| 13262/13262 [04:39<00:00, 47.50it/s]


In [77]:
list_data_label_info[0]

[(3355, '他', 'label_O', 'label_O', 'label_O', 'label_O', 'r', 'S', 'B', 'E'),
 (3355, '告', 'label_O', 'label_O', 'label_O', 'label_O', 'v', 'B', 'B', 'I'),
 (3355, '诉', 'label_O', 'label_O', 'label_O', 'label_O', 'v', 'E', 'I', 'E'),
 (3355,
  '记',
  'label_B_position',
  'label_B_position',
  'label_I_position',
  'label_I_position',
  'n',
  'B',
  'B',
  'I'),
 (3355,
  '者',
  'label_E_position',
  'label_I_position',
  'label_E_position',
  'label_I_position',
  'n',
  'E',
  'I',
  'E'),
 (3355, '，', 'label_O', 'label_O', 'label_O', 'label_O', 'x', 'S', 'B', 'E'),
 (3355, '自', 'label_O', 'label_O', 'label_O', 'label_O', 'r', 'B', 'B', 'I'),
 (3355, '己', 'label_O', 'label_O', 'label_O', 'label_O', 'r', 'E', 'I', 'E'),
 (3355, '在', 'label_O', 'label_O', 'label_O', 'label_O', 'p', 'S', 'B', 'E'),
 (3355,
  '中',
  'label_B_address',
  'label_B_address',
  'label_I_address',
  'label_I_address',
  'ns',
  'B',
  'B',
  'I'),
 (3355,
  '山',
  'label_I_address',
  'label_I_address',
  'l

# 增强数据

In [81]:
def get_dataframe(path_dir_data):
    # 获取的数据
    list_ID = []
    list_data = []
    
    # 获取ID
    list_data_ID = [file_name.split('.')[0] for file_name in os.listdir(path_dir_data) if '.txt' in file_name]
    # 循环ID
    for ID in tqdm(set(list_data_ID)):
        # 获取词
        data_path = os.path.join(path_dir_data,ID+'.txt')
        
        with open(data_path,'r',encoding="utf-8") as f:
            line = ''.join(f.readlines())
        # 获取词
        list_ID += [ID]
        list_data += [line]
    # dataframe
    df_data = pd.DataFrame()
    df_data['ID'] = list_ID
    df_data['data'] = list_data
    return df_data

In [82]:
path_dir_data_train = './data/train_data/'
path_dir_data_test = './data/test_data/'

In [83]:
df_train = get_dataframe(path_dir_data_train)

100%|██████████████████████████████████████████████████████████████████████████| 13262/13262 [00:01<00:00, 7160.38it/s]


In [84]:
df_test = get_dataframe(path_dir_data_test)

100%|█████████████████████████████████████████████████████████████████████████████| 3956/3956 [00:26<00:00, 151.22it/s]


In [85]:
df_train

,ID,data
0,3355,他告诉记者，自己在中山公园上班，特意赶在上班前到银行来办理放贷业务。说起七折优惠，王先生一肚...
1,2835,大师”，知名星际主持人，常年解说SPL，MSL，OSL，PLU举办的比赛，
2,4641,均为正正负形态，切沃vs罗马一场客胜盈亏最低，赢球概率最大，不过目前来看，3场都可以作博胆。
3,1451,对两强1胜1平使奥尼尔尝到了甜头，也不再一味坚持他用惯了的433，正是上个月2：1主场力克皇...
4,6649,京开高速方便畅达到达金融街、三里河、丽泽商务区；。
...,...,...
13257,5505,最好建议小雨本人去长沙就近的派出所报警。公安回应已开展工作29日，@南昌铁路公安局回应
13258,6892,数据显示，昨天上海黄金交易所黄金au99.95、黄金au99.99、黄金au100g、黄金a...
13259,7125,日前，静安法院一审判决黄春败诉。
13260,8184,saidintibazonkiza，前锋，布隆迪）、比科林（jhonnyvan


In [86]:
df_test

,ID,data
0,1288,中国陶瓷艺术大师，广东省工艺美术协会会员，石湾公仔协会副会长，鸿之陶美术工艺厂负责人。工作事...
1,3355,标语书法家、车窗摄影家、低级政治家、无股资本家、搭车旅行家、茅庐建筑师、乡村规划师、贫农园艺...
2,242,韩国文化部突然将星际争霸2化为18禁游戏。具体原因：要知道目前凯尔特人还保持着欧冠赛场最长的...
3,3184,香港动漫画联会会长、漫画家。成名作品<龙虎门><天子传奇>＜神兵玄奇＞，总销量超过四亿本。动...
4,1568,《21世纪》：近期，您指出，市场风险、房地产市场风险和信用风险是宏观监管需要关注的重要方面。
...,...,...
3951,2756,无缘大慈同体大悲。愿我们所有的人都能生慈悲心断恶修善吃素放生念佛忏悔愿大家诸事圆满。阿弥陀佛...
3952,573,你是秋天里的风，我却是一片叶。当你来到我的身边，我跟随着你。而你不会停留，吹向远方，一直向前...
3953,3408,风神A60，大气浑厚外观，MR系列2.0排量发动机与CVT无极变速器，2700mm超长轴距与...
3954,2009,两支排名靠后的球队交手，更看好主场作战的马洛卡，毕竟主队连续两个赛季的主场都击败了维尔瓦，k...


In [87]:
def get_label(path_dir_data):
    # 获取的数据
    list_df_label = []
    
    # 获取ID
    list_label_ID = [file_name.split('.')[0] for file_name in os.listdir(path_dir_label) if '.csv' in file_name]
    # 循环ID
    for ID in tqdm(set(list_label_ID)):
        
        # 获取标签
        label_path = os.path.join(path_dir_label,ID+'.csv')
        
        df_label = get_df_label(label_path)
        
        list_df_label += [df_label]
        

    # dataframe
    df_label = pd.concat(list_df_label)
    return df_label

In [88]:
df_label_train = get_label('./data/train_label/')

100%|███████████████████████████████████████████████████████████████████████████| 13262/13262 [00:24<00:00, 538.97it/s]


In [89]:
df_label_train.head()

,ID,Category,Pos_b,Pos_e,Privacy
0,3355,address,9,12,中山公园
1,3355,position,3,4,记者
0,2835,game,6,7,星际
1,2835,organization,16,18,SPL
2,2835,position,8,10,主持人


In [90]:
set(df_label_train.Category)

{'QQ',
 'address',
 'book',
 'company',
 'email',
 'game',
 'government',
 'mobile',
 'movie',
 'name',
 'organization',
 'position',
 'scene',
 'vx'}

In [91]:
df_label_train_Category = df_label_train.groupby(['Category'])['Privacy'].agg(lambda x: ' '.join([str(i) for i in list(x)])).reset_index()
df_label_train_Category

,Category,Privacy
0,QQ,1827967769 18618193311 371451138 783594553 533...
1,address,中山公园 金融街 台北 oldlauncestonseaport 越南 堪萨斯城 摩天大厦 ...
2,book,《老子·五十八章》 《红楼梦》 《梅冈城故事》 抗日之铁血兵王 晨报 乾隆御制诗 《故城时光...
3,company,光大银行深圳分行信用卡中心 中国银联 中资银行 航天科技中心 EA 中华油脂网 金鹰网 中行...
4,email,yinentertainment@gmail.com guanshaozeng007@163...
5,game,星际 星际争霸 《魔兽世界》 《古墓丽影：传奇》 WAR3 DotA 魔兽争霸3 StarC...
6,government,石家庄市政府 马达加斯加政府 最高法 美国海军陆战队 国务院 美国海军 俄空军 广州国土房管...
7,mobile,+85223352105 18911738839 886926929796 +8526687...
8,movie,《好家伙坏家伙怪家伙》 30DaysofNight 《射雕英雄传》 《姨妈的后现代生活》 《...
9,name,奥尼尔 老子 董玉辉 易纲 奈史密斯 郭清保 史蒂夫奈尔斯 翁美玲 CCM. Grigoro...


In [92]:
df_label_train_Category['Category_num'] = df_label_train_Category['Privacy'].map(lambda x:len(x.split(' ')))

In [93]:
df_label_train_Category.sort_values(['Category_num'])

,Category,Privacy,Category_num
13,vx,1827967769 waveapp songxiaobo montager rachel-...,19
0,QQ,1827967769 18618193311 371451138 783594553 533...,28
4,email,yinentertainment@gmail.com guanshaozeng007@163...,259
7,mobile,+85223352105 18911738839 886926929796 +8526687...,277
2,book,《老子·五十八章》 《红楼梦》 《梅冈城故事》 抗日之铁血兵王 晨报 乾隆御制诗 《故城时光...,1642
12,scene,苏州博物馆 太空体验基地 瓜岛 西本愿寺 故宫 泸沽湖 维多利亚花园 阿里山森铁 太湖 念青...,1936
6,government,石家庄市政府 马达加斯加政府 最高法 美国海军陆战队 国务院 美国海军 俄空军 广州国土房管...,2648
8,movie,《好家伙坏家伙怪家伙》 30DaysofNight 《射雕英雄传》 《姨妈的后现代生活》 《...,2680
5,game,星际 星际争霸 《魔兽世界》 《古墓丽影：传奇》 WAR3 DotA 魔兽争霸3 StarC...,3468
1,address,中山公园 金融街 台北 oldlauncestonseaport 越南 堪萨斯城 摩天大厦 ...,4146


## QQ 数据产生

In [94]:
from collections import Counter
import random

In [97]:
txt_QQ = df_label_train_Category[df_label_train_Category['Category']=='QQ']['Privacy'].to_list()[0]
txt_QQ

'1827967769 18618193311 371451138 783594553 53344829 130238288 114281231 2308713823 2450006972 415392980 326597719 232297193 89359735 1787817738 14705257 496049279 229215757 490401933 527622886 160308912 826929452 776979804 33370722 1941600020 924969718 449801320 574785760 1143175993'

In [98]:
Counter([len(i) for i in txt_QQ.split(' ')])

Counter({10: 6, 11: 1, 9: 17, 8: 4})

In [102]:
def get_qq():
    qq_len = random.randint(8,11)
    qq = ''
    for i in range(qq_len):
        qq += str(random.randint(0,9))
    return qq

In [103]:
get_qq()

'74391500'

# 数据增强

In [104]:
!ls ../data/train/data/

'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [105]:
def get_df_label(label_path):
    return pd.read_csv(label_path)

In [109]:
def get_list_data(data_path):
    with open(data_path,'r',encoding="utf-8") as f:
        line = ''.join(f.readlines())
        list_data = list(line)
    return list_data

In [110]:
df_label = get_df_label('./data/train_label/0.csv')

In [111]:
list_data = get_list_data('./data/train_data/0.txt')

In [112]:
def get_BIOES(list_data, df_label):
    list_label = ['O'] * len(list_data)
    for index,d in df_label.iterrows():
        ID = d['ID']
        Category = d['Category']
        Pos_b = d['Pos_b']
        Pos_e = d['Pos_e']
        
        # 异常数据处理
        if 2162 == ID:
            Category == '前明骏女孩组合队长"'
            Pos_e = Pos_e - 1

        if Pos_b == Pos_e:
            list_label[Pos_b] = 'S_' + Category
        elif Pos_b - Pos_e == 1:
            list_label[Pos_b] = 'B_' + Category
            list_label[Pos_e] = 'E_' + Category
        else:
            # 异常数据处理
            try:
                list_label[Pos_b] = 'B_' + Category
                list_label[Pos_e] = 'E_' + Category 
                for pos_i in range(Pos_b+1,Pos_e):
                    list_label[pos_i] = 'I_' + Category 
            except:
                print(ID)
    return [(ID,data,label) for data,label in zip(list_data,list_label)]

In [113]:
BIOES = get_BIOES(list_data, df_label)
BIOES[0]

(0, '艺', 'O')

# 方法一 插入 产生数据

In [115]:
def get_qq():
    qq_len = random.randint(8,10)
    qq = ''
    for i in range(qq_len):
        qq += str(random.randint(0,9))
    return qq

In [116]:
def get_BIOES_qq(qq,ID):
    list_qq = list(qq)
    Pos_b, Pos_e = 0, len(list_qq)-1
    
    list_label = ['O'] * len(list_qq)
    if Pos_b == Pos_e:
        list_label[Pos_b] = 'S_' + 'QQ'
    elif Pos_e - Pos_b == 1:
        list_label[Pos_b] = 'B_' + 'QQ'
        list_label[Pos_e] = 'E_' + 'QQ'
    else:
        list_label[Pos_b] = 'B_' + 'QQ'
        list_label[Pos_e] = 'E_' + 'QQ'
        for pos_i in range(Pos_b+1,Pos_e):
            list_label[pos_i] = 'I_' + 'QQ'
    return [(ID,data,label) for data,label in zip(list_qq,list_label)]

In [117]:
get_BIOES_qq(get_qq(),16)

[(16, '8', 'B_QQ'),
 (16, '0', 'I_QQ'),
 (16, '6', 'I_QQ'),
 (16, '3', 'I_QQ'),
 (16, '6', 'I_QQ'),
 (16, '0', 'I_QQ'),
 (16, '8', 'I_QQ'),
 (16, '3', 'E_QQ')]

In [121]:
def get_new_data(list_data, df_label):
    #ID,data,label
    #(0, '艺', 'O')
    BIOES = get_BIOES(list_data, df_label)
    ID = BIOES[0][0]
    #ID,data,label
    #(16, '3', 'E_QQ')
    qq_BIOES = get_BIOES_qq(get_qq(),ID)
    print(qq_BIOES)
    # 随机插入 O 的位置
    len_BIOES = len(BIOES)
    r_loc = random.randint(0,len(BIOES))
    if r_loc == 0:
        augument_BIOES = qq_BIOES + BIOES
    elif r_loc == len_BIOES:
        augument_BIOES = BIOES + qq_BIOES
    else:
        augument_BIOES = BIOES
        for r_loc_i in range(r_loc, len_BIOES):
            if BIOES[r_loc_i][-1] == 'O':
                augument_BIOES = BIOES[:r_loc_i] + qq_BIOES + BIOES[r_loc_i:]  
                break
    return augument_BIOES

In [122]:
get_new_data(list_data, label)

[(0, '1', 'B_QQ'), (0, '2', 'I_QQ'), (0, '4', 'I_QQ'), (0, '1', 'I_QQ'), (0, '5', 'I_QQ'), (0, '8', 'I_QQ'), (0, '6', 'I_QQ'), (0, '8', 'I_QQ'), (0, '3', 'I_QQ'), (0, '4', 'E_QQ')]


[(0, '艺', 'O'),
 (0, '术', 'O'),
 (0, '是', 'O'),
 (0, '相', 'O'),
 (0, '同', 'O'),
 (0, '的', 'O'),
 (0, '，', 'O'),
 (0, '音', 'O'),
 (0, '乐', 'O'),
 (0, '美', 'O'),
 (0, '术', 'O'),
 (0, '体', 'O'),
 (0, '育', 'O'),
 (0, '三', 'O'),
 (0, '样', 'O'),
 (0, '都', 'O'),
 (0, '是', 'O'),
 (0, '艺', 'O'),
 (0, '术', 'O'),
 (0, '。', 'O'),
 (0, '，', 'O'),
 (0, '三', 'O'),
 (0, '样', 'O'),
 (0, '艺', 'O'),
 (0, '术', 'O'),
 (0, '都', 'O'),
 (0, '是', 'O'),
 (0, '靠', 'O'),
 (0, '感', 'O'),
 (0, '觉', 'O'),
 (0, '的', 'O'),
 (0, '。', 'O'),
 (0, '感', 'O'),
 (0, '觉', 'O'),
 (0, '好', 'O'),
 (0, '玩', 'O'),
 (0, '1', 'B_QQ'),
 (0, '2', 'I_QQ'),
 (0, '4', 'I_QQ'),
 (0, '1', 'I_QQ'),
 (0, '5', 'I_QQ'),
 (0, '8', 'I_QQ'),
 (0, '6', 'I_QQ'),
 (0, '8', 'I_QQ'),
 (0, '3', 'I_QQ'),
 (0, '4', 'E_QQ'),
 (0, '起', 'O'),
 (0, '来', 'O'),
 (0, '就', 'O'),
 (0, '很', 'O'),
 (0, '轻', 'O'),
 (0, '松', 'O'),
 (0, '，', 'O'),
 (0, '所', 'O'),
 (0, '以', 'O'),
 (0, '叫', 'O'),
 (0, '做', 'O'),
 (0, '玩', 'O'),
 (0, '艺', 'O'),
 (0, '术', 'O'),
 (0, '。', 

# 工具包

In [173]:
# !pip install nlpcda

In [20]:
from nlpcda import Randomword

Using TensorFlow backend.


In [24]:
test_str = '''这是个实体：58同城；今天是2020年3月8日11:40，天气晴朗，天气很不错，空气很好，不差；这个nlpcad包，用于方便一键数据增强，可有效增强NLP模型的泛化性能、减少波动、抵抗对抗攻击'''

smw = Randomword(create_num=4,change_rate=5)
rs1 = smw.replace(test_str)

print('随机实体替换>>>>>>')
for s in rs1:
    print(s)

load :C:\ProgramData\Anaconda3\lib\site-packages\nlpcda\data\company.txt done
随机实体替换>>>>>>
这是个实体：58同城；今天是2020年3月8日11:40，天气晴朗，天气很不错，空气很好，不差；这个nlpcad包，用于方便一键数据增强，可有效增强NLP模型的泛化性能、减少波动、抵抗对抗攻击
这是个实体：中国通信服务；今天是2020年3月8日11:40，天气晴朗，天气很不错，空气很好，不差；这个nlpcad包，用于方便一键数据增强，可有效增强NLP模型的泛化性能、减少波动、抵抗对抗攻击
这是个实体：瑞丰光电；今天是2020年3月8日11:40，天气晴朗，天气很不错，空气很好，不差；这个nlpcad包，用于方便一键数据增强，可有效增强NLP模型的泛化性能、减少波动、抵抗对抗攻击


In [175]:
Randomword?

In [176]:
!cat /home/work/miniconda3/envs/pycookly2/lib/python3.6/site-packages/nlpcda/data/company.txt

国投集团控股
银河娱乐
金界控股
均安控股
泛海酒店
德祥地产
亚洲能源物流
搜房网
中国电信
弘阳地产
昱辉阳光
领航医药生物科技
宇华教育
龙翔集团
国际家居零售
钧濠集团
港铁公司
KAKIKO GROUP
红黄蓝
万科企业
宏安地产
科兴生物
南旋控股
中信证券
迪臣发展国际
创业集团控股
凌锐控股
BOC 2014 PREF
云能国际
中国石油化工股份
进升集团控股
汇汉控股
东瀛游
联合能源集团
数码通电讯
元征科技
药明生物
HKE HOLDINGS
宏华集团
德基科技控股
趣头条
康宁医院
天保能源
58同城
中国新城市
中国兴业控股
华宝国际
福森药业
嘉进投资国际
远东控股国际
国瑞置业
惠生工程
冠力国际
九兴控股
中国服饰控股
远东环球
龙辉国际控股
中国联通(香港)
同方友友
中国香精香料
仁瑞投资
雨润食品
中国绿岛科技
中国汽车物流
华昱高速
恒益控股
STYLAND HOLD
中国人寿
旭明光电
和信贷
麦盛资本
九洲大药房
华显光电
盈利时
正业国际
东方报业集团
云锋金融
中国水业集团
环球实业科技
中油洁能控股
东方海外国际
绿色能源科技集团
复锐医疗科技
CEC INT'L HOLD
新鸿基地产
昆仑国际金融
信佳国际
英利
威华达控股
高力集团
汉国置业
盛京银行
青岛啤酒股份
中国石油股份
正恒国际控股
同方康泰
三盛控股
先丰服务集团
现恒建筑
中石化冠德
中环球船务
优品３６０
天安
思城控股
中国铝罐
富通科技
工商银行
正干金融控股
华能电力
珠江船务
同得仕（集团）
金嗓子
寺库
百富环球
阿特斯
融信中国
联亚集团
中国山东高速金融
乙德投资控股
广泰国际控股
复星旅游文化
星美文化旅游
华润燃气
汇付天下
景福集团
富阳
中国北大荒
中国南方航空股份
兴利（香港）控股
佳兆业健康
信阳毛尖
杉杉品牌
达内科技
沿海家园
比速科技
北控清洁能源集团
精优药业
中国财险
丰德丽控股
百济神州
中国儿童护理
桐成控股
美亨实业
大生农业金融
五矿地产
联合地产（香港）
结好金融
中移动
迪信通
洛阳玻璃股份
ＢＢＩ生命科学
浦林成山
HERALD HOLD
北京控股
新鸿基公司
大成生化科技
前海健康
建生国际
合和公路基建—Ｒ
旅业国际
陆氏集团（越南）
上为
金源米业
创科实业
同佳健康
民银资本
大唐西市
成

In [194]:
2

2

# 模型

In [255]:
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer

In [256]:
import tensorflow as tf

In [258]:
build_transformer_model?

In [259]:
!cat ~/miniconda3/envs/pycookly2/lib/python3.6/site-packages/bert4keras/models.py

#! -*- coding: utf-8 -*-
# 主要模型

import numpy as np
from bert4keras.layers import *
from bert4keras.snippets import delete_arguments
from keras.models import Model
import json


class Transformer(object):
    """模型基类
    """
    def __init__(
        self,
        vocab_size,  # 词表大小
        hidden_size,  # 编码维度
        num_hidden_layers,  # Transformer总层数
        num_attention_heads,  # Attention的头数
        intermediate_size,  # FeedForward的隐层维度
        hidden_act,  # FeedForward隐层的激活函数
        dropout_rate=None,  # Dropout比例
        embedding_size=None,  # 是否指定embedding_size
        attention_key_size=None,  # Attention中Q,K的head_size
        sequence_length=None,  # 是否固定序列长度
        keep_tokens=None,  # 要保留的词ID列表
        layers=None,  # 外部传入的Keras层
        name=None,  # 模型名称
        **kwargs
    ):
        if keep_tokens is None:
            self.vocab_size = vocab_size
        else:
            self.vocab_size = len(keep_tokens)
        self.hidden_size = hidden_size
        self.nu

In [186]:
!cat ~/miniconda3/envs/pycookly2/lib/python3.6/site-packages/bert4keras/models.py

#! -*- coding: utf-8 -*-
# 主要模型

import numpy as np
from bert4keras.layers import *
from bert4keras.snippets import delete_arguments
from keras.models import Model
import json


class Transformer(object):
    """模型基类
    """
    def __init__(
        self,
        vocab_size,  # 词表大小
        hidden_size,  # 编码维度
        num_hidden_layers,  # Transformer总层数
        num_attention_heads,  # Attention的头数
        intermediate_size,  # FeedForward的隐层维度
        hidden_act,  # FeedForward隐层的激活函数
        dropout_rate=None,  # Dropout比例
        embedding_size=None,  # 是否指定embedding_size
        attention_key_size=None,  # Attention中Q,K的head_size
        sequence_length=None,  # 是否固定序列长度
        keep_tokens=None,  # 要保留的词ID列表
        layers=None,  # 外部传入的Keras层
        name=None,  # 模型名称
        **kwargs
    ):
        if keep_tokens is None:
            self.vocab_size = vocab_size
        else:
            self.vocab_size = len(keep_tokens)
        self.hidden_size = hidden_size
        self.nu